# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [196]:
import pandas as pd
import numpy as np

## imports for Python, Pandas

In [197]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [198]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [ ]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

In [ ]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [ ]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

In [ ]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

## JSON Mini Project Solutions

In [264]:
world_bank_projects_df = pd.read_json('data/world_bank_projects.json')

#### 1. Find the 10 countries with most projects

In [204]:
world_bank_projects_df.groupby('countryname').size().sort_values(ascending=False).head(10)

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
dtype: int64

#### 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

In [219]:
#extract mjtheme_namecode column
project_themes_df = world_bank_projects_df[['mjtheme_namecode']]

#Convert dataframe to dict object to be used in json_normalize
project_themes_dict = project_themes_df.to_dict()

#Convert the dict of dicts to dataframe by using json_normalize
theme_namecode_df = pd.DataFrame()
for i in project_themes_dict['mjtheme_namecode']:
    theme_namecode_df = theme_namecode_df.append(json_normalize(project_themes_dict['mjtheme_namecode'][i]), ignore_index=True)

#Replace empty values under "name" column with NaN
theme_namecodeNaN_df = theme_namecode_df.replace(r'^\s*$', np.nan, regex=True)

#sort and fill missing values with last valid "name" 
theme_namecode_df = theme_namecodeNaN_df.sort_values(['code', 'name']).ffill()

#Groupby and get the top 10 major project themes, Note: size() includes NaN values
theme_namecode_df.groupby(['name']).name.size().sort_values(ascending=False).head(10)


name
Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                           77 
Urban development                               50 
Economic management                             38 
Name: name, dtype: int64

#### 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [266]:
print(world_bank_projects_df['mjtheme_namecode'].head(10))
#count the missing values in name column
print()
print('Total number of missing names: %d' %theme_namecodeNaN_df['name'].isna().sum())
#This missing names comes from original dataset 'mjtheme_namecode' column, which needs to be cleaned

0    [{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]                                                                                                                                                                   
1    [{'code': '1', 'name': 'Economic management'}, {'code': '6', 'name': 'Social protection and risk management'}]                                                                                                                             
2    [{'code': '5', 'name': 'Trade and integration'}, {'code': '2', 'name': 'Public sector governance'}, {'code': '11', 'name': 'Environment and natural resources management'}, {'code': '6', 'name': 'Social protection and risk management'}]
3    [{'code': '7', 'name': 'Social dev/gender/inclusion'}, {'code': '7', 'name': 'Social dev/gender/inclusion'}]                                                                                                                               
4    [{'code': '5', 'name': 'Trade a

In [267]:
#Store unique code-name pairs in a dictionary 
theme_namecode = theme_namecodeNaN_df.drop_duplicates().dropna()
theme_namecode.set_index('code',inplace=True)
theme_namecode_dict = theme_namecode.to_dict()
theme_namecode_dict = theme_namecode_dict['name']

#Replace empty names with appropriate names from the code-name pair dictionary
for row in world_bank_projects_df['mjtheme_namecode']:
    for d in row:
        if d['name'] == '':
            d['name'] = theme_namecode_dict[d['code']]

print(world_bank_projects_df['mjtheme_namecode'].head(10))

0    [{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': 'Environment and natural resources management'}]                                                                                                                       
1    [{'code': '1', 'name': 'Economic management'}, {'code': '6', 'name': 'Social protection and risk management'}]                                                                                                                             
2    [{'code': '5', 'name': 'Trade and integration'}, {'code': '2', 'name': 'Public sector governance'}, {'code': '11', 'name': 'Environment and natural resources management'}, {'code': '6', 'name': 'Social protection and risk management'}]
3    [{'code': '7', 'name': 'Social dev/gender/inclusion'}, {'code': '7', 'name': 'Social dev/gender/inclusion'}]                                                                                                                               
4    [{'code': '5', 'name': 'Trade a